<a href="https://colab.research.google.com/github/RL-Starterpack/rl-starterpack/blob/main/exercises/AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RL Tutorial - **Bandits Exercise**


## Setup

In [ ]:
#@title Run this cell to clone the RL tutorial repository and install it
try:
  import rl_starterpack
  print('RL-Starterpack repo succesfully installed!')
except ImportError:
  print('Cloning RL-Starterpack package...')

  # !git clone https://github.com/RL-Starterpack/rl-starterpack.git
  print('Installing RL-StarterPack package...')
  !pip install -e rl-starterpack[full] &> /dev/null
  print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('Please restart the runtime to use the newly installed package!')
  print('Runtime > Restart Runtime')
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
from typing import List, Optional, Tuple, Union
from IPython.display import Markdown as md
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from functools import partial
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid
sns.set()
%matplotlib inline

As a first step, let's do some experimentation setup. The crucial thing we need is a **test bed**. The test bed is the object where we define the arms, the reward distributions for each arm and the operation of drawing an arm and returning the ensuing reward.

We will provide some parameters to play with. Specifically, we can change the number of arms and the parameters of the distribution from which the mean reward for each arm is sampled.

We will also define the sampling functions that specify how the mean rewards for each arm is assigned and how the observed rewards are randomly sampled.

In [ ]:
# PARAMETERS
num_arms = 10               # number of arms in the test bed
reward_mean = 1             # mean of normal distribution from which mean reward of each arm will be drawn
reward_std = 1              # std of normal distribution from which mean reward of each arm will be drawn
                            # we will keep the std fixed throughout

def arm_mean_sampler(size: int=1):
    return np.random.normal(loc=reward_mean, scale=reward_std, size=size)

def arm_reward_sampler(mu: float, size: int=1):
    reward = np.random.normal(loc=mu, scale=reward_std, size=size)
    if size == 1:
        return reward[0]
    return reward

class TestBed:

    def __init__(self,
                num_arms: int=10,
                arm_mean_sampler: callable=arm_mean_sampler,
                arm_reward_sampler: callable=arm_reward_sampler) -> None:
        self.seed = np.random.choice(1000000)
        self.arm_mean_sampler = arm_mean_sampler
        self.arm_reward_sampler = arm_reward_sampler
        self.num_arms = num_arms

        np.random.seed(self.seed)
        self.mean_rewards = arm_mean_sampler(size=num_arms)

    def draw_arm(self, idx: int) -> float:
        mu = self.mean_rewards[idx]
        return self.arm_reward_sampler(mu)

    def visualise(self) -> None:
        dummies = np.random.normal(0, 1, 1000)
        dfs = []
        for arm, mu in enumerate(self.mean_rewards):
            dfs.append(pd.DataFrame({'Arm': arm, 'Reward Distribution': dummies + mu}))

        dfs = pd.concat(dfs)
        fig, ax = plt.subplots(figsize=(10, 5))
        sns.violinplot(data=dfs, x='Arm', y='Reward Distribution', width=0.3, ax=ax)
        plt.show()


### The Testbed

The testbed can be visualised with `testbed.visualise()` to check the distribution of rewards for each arm in the testbed. Keep a note of the arm with the highest actual mean reward. Ideally, we would like each of the policies we implement to find this optimal arm.

In [ ]:
testbed = TestBed(num_arms=num_arms, arm_mean_sampler=arm_mean_sampler, arm_reward_sampler=arm_reward_sampler)
testbed.visualise()

md(f'In the testbed above, the **optimal arm** is arm **{testbed.mean_rewards.argmax()}**')

In [ ]:
class Experiment:
    
    def __init__(self, testbed: TestBed, policy: callable, **parameters) -> None:
        self.testbed = testbed
        self.policy = policy
        self.init_rewards = parameters.get('init_rewards', 0)
        self._init_fields()

    def _init_fields(self) -> None:
        self.observed_rewards = []
        self.expected_rewards = self.init_rewards * np.ones(self.testbed.num_arms)
        self.observed_draws = []
        self.num_draws = np.zeros(self.testbed.num_arms, dtype=np.int32)
        self.total_draws = 0

    def step(self) -> Tuple[int, float]:
        chosen_arm = self.policy(self.expected_rewards, self.num_draws)
        reward = self.testbed.draw_arm(chosen_arm)

        self.total_draws += 1
        self.observed_draws.append(chosen_arm)
        self.num_draws[chosen_arm] += 1
        self.observed_rewards.append(reward)
        self.expected_rewards[chosen_arm] += (
            (1. / (self.num_draws[chosen_arm] + 1)) * (reward - self.expected_rewards[chosen_arm]))

        return chosen_arm, reward

    def run_episode(self, num_steps: int) -> None:
        for i in range(num_steps):
            _ = self.step()

    def reset(self) -> None:
        self._init_fields()

    def visualise(self, kind='line') -> None:
        fig, ax = plt.subplots(figsize=(15, 10), nrows=3, ncols=1)

        reward_df = pd.DataFrame({
                            'timestep': range(len(self.observed_rewards)),
                            'reward': self.observed_rewards})
        if kind=='line':
            sns.lineplot(data=reward_df, x='timestep', y='reward', ax=ax[0])
        else:
            sns.scatterplot(data=reward_df, x='timestep', y='reward', hue='reward', ax=ax[0])

        sns.violinplot(x=self.observed_draws, y=self.observed_rewards, width=0.5, ax=ax[1])
        ax[1].set_ylabel('reward')
        ax[1].set_xlabel('arm')

        arms, counts = np.unique(self.observed_draws, return_counts=True)
        sns.scatterplot(x=arms, y=counts, ax=ax[2])
        ax[2].set_ylim(0, len(self.observed_rewards))
        ax[2].set_ylabel('draws')
        ax[2].set_xlabel('arm')

        plt.show()



def average_experiments(policy: callable,
                        init_rewards: Union[int, float]=0,
                        timesteps: int=1000, repeats: int=200,
                        **parameters) -> None:
    for k, v in parameters.items():
        if not isinstance(v, (list, tuple)):
            parameters[k] = [v]

    parameters = ParameterGrid(parameters)
    results = []
    mean_rewards = []
    optimal_arm_draws = []


    for param_set in parameters:
        param_set_string = ', '.join(f'{k}={v}' for k, v in param_set.items())
        observed_rewards = []
        for n in tqdm(range(repeats)):
            testbed = TestBed()
            optimal_arm = testbed.mean_rewards.argmax()
            policy = partial(policy, **param_set)
            experiment = Experiment(testbed=testbed, policy=policy, **param_set)
            experiment.run_episode(num_steps=timesteps)
            observed_rewards.append(experiment.observed_rewards)
            optimal_arm_draws.append((n, experiment.num_draws[optimal_arm], param_set_string))
            mean_rewards.append((n, np.array(experiment.observed_rewards).mean(), param_set_string))
        
        observed_rewards = np.array(observed_rewards).T.flatten()
        rewards_df = pd.DataFrame({
                                'timestep': np.repeat(range(timesteps), repeats),
                                'reward': observed_rewards,
                                'params': param_set_string
                    })
        results.append(rewards_df)
    
    fig, ax = plt.subplots(figsize=(15, 10), nrows=3, ncols=1)

    results = pd.concat(results)
    sns.lineplot(data=results,
                x='timestep',
                y='reward',
                hue='params',
                ax=ax[0])
    mean_rewards = pd.DataFrame(mean_rewards, columns=['iteration', 'mean reward', 'params'])
    sns.violinplot(data=mean_rewards,
                x='params',
                y='mean reward',
                ax=ax[1])
    optimal_arm_draws = pd.DataFrame(optimal_arm_draws, columns=['iteration', 'optimal draws', 'params'])
    sns.violinplot(data=optimal_arm_draws,
                x='params',
                y='optimal draws',
                ax=ax[2])
    plt.show()

## The EXPERIMENT

The `Experiment` object will help us run and accumulate the results of a sequence of arm pulls of a testbed. We will provide a **testbed** and a **policy** to the **experiment**. We will pull arms from the testbed using the policy for a fixed number of timesteps. After that we can visualise the results. Specifically, we will look at the profile of rewards received over time according to the policy, the distribution of rewards observed from each arm and the number of times each arm was pulled in the experiment. To quantify the quality of the policies we implement we will also look the optimal arm found by each policy and the average reward accumulated over the experiment's time horizon.

## POLICIES

### Explore Then Commit
In this policy we systematically explore each arm for a fixed number of rounds. After that, we exploit the best known arm for the remaining timesteps.

The hyperparameter for this policy will be the **number of rounds** for which we explore the arms. We can set this value to be anything between `1` (explore the arms for one round only) and `horizon / number_of_arms` (keep exploring the arms throughout).

Set the hyperparameter and implement the policy in the following cell.

In [ ]:
# Implement policy here
num_rounds = 50

def explore_then_commit(expected_rewards: List[float], num_draws: List[int], num_rounds: int=num_rounds) -> int:
    num_arms = len(expected_rewards)
    total_draws = sum(num_draws)
    if total_draws < num_rounds * num_arms:
        return total_draws % num_arms

    max_reward_arms = np.where(expected_rewards == max(expected_rewards))[0]
    return np.random.choice(max_reward_arms)

# run experiment with policy
experiment = Experiment(testbed, explore_then_commit)
experiment.run_episode(num_steps=1000)
experiment.visualise()

md(f'''
Optimal arm found with **ETC** is **{experiment.expected_rewards.argmax()}**

Number of times each arm was **pulled**: {dict(zip(range(len(experiment.num_draws)), experiment.num_draws))}

**Average reward** collected is **{np.array(experiment.observed_rewards).mean()}**
''')

In [ ]:
average_experiments(explore_then_commit, num_rounds=[25, 50])

### $\epsilon$-greedy
In this policy, we either uniformly choose a random arm with a probability $ \epsilon \in (0,1)$ or choose the arm with best known expected reward.

In [ ]:
epsilon = 0.1

# IMPLEMENT policy here
def epsilon_greedy(expected_rewards: List[float], num_draws: List[int], epsilon: float=epsilon) -> int:
    if np.random.uniform() <= epsilon:
        choices = range(len(expected_rewards))
    else:
        choices = np.where(expected_rewards == max(expected_rewards))[0]
        
    arm = np.random.choice(choices)
    return arm

# run experiment with policy
experiment = Experiment(testbed, epsilon_greedy)
experiment.run_episode(num_steps=1000)
experiment.visualise()

md(f'''
Optimal arm found with **$\epsilon$-greedy** is **{experiment.expected_rewards.argmax()}**

Number of times each arm was **pulled**: {dict(zip(range(len(experiment.num_draws)), experiment.num_draws))}

**Average reward** collected is **{np.array(experiment.observed_rewards).mean()}**
''')

In [ ]:
average_experiments(epsilon_greedy, epsilon=[0.01, 0.1, 1])

### Optimistic Initial Values (**OIV**)
In this policy, we set the initial expected reward values to a **wildly optimistic high number**. The reasoning is, as we update the high initial expected rewards with the observed rewards, they will tend to lower values. In this setting, if we choose the arm with the highest expected reward, we will end up exploring the arms more until the arm with the best reward is found.

In [ ]:
optimistic_initial_reward = 5

# IMPLEMENT policy here
def optimistic_initial_values(expected_rewards: List[float], num_draws: int) -> int:
    max_reward_arms = np.where(expected_rewards == max(expected_rewards))[0]
    return np.random.choice(max_reward_arms)

# run experiment with policy
experiment = Experiment(testbed, optimistic_initial_values, init_rewards=optimistic_initial_reward)
experiment.run_episode(num_steps=1000)
experiment.visualise()

md(f'''
Optimal arm found with **OIV** is **{experiment.expected_rewards.argmax()}**

Number of times each arm was **pulled**: {dict(zip(range(len(experiment.num_draws)), experiment.num_draws))}

**Average reward** collected is **{np.array(experiment.observed_rewards).mean()}**
''')

In [ ]:
average_experiments(optimistic_initial_values, init_rewards=[1, 5, 10])

### Upper Confidence Bound (**UCB**)
In this policy, we use the number of times each arm has been drawn to compute a confidence interval for the expected reward for each arm. At each step we select the arm with the highest upper bound for the confidence interval. The **weight given to uncertainty term**, $c \in (0, 1]$ is a hyperparameter in this case.

This method is the most popular policy used for bandits. The uncertainty bounds aid is exploration in the initial stages, but stops it as soon as soon as the optimal arm is found.

One thing to keep in mind is UCB can only be applied after each arm has been tried at least once. This is to ensure that uncertainty for each arm can be computed. For this we add another hyperparameter for the **number of tries**. Until each arm has been pulled this minimum number of times, we randomly select one of the arms that has not yet been pulled enough times.

In [ ]:
# IMPLEMENT policy here
weight = 0.5
num_tries_per_arm = 10

def upper_confidence_bound(expected_rewards: List[float],
                        num_draws: List[int],
                        weight: float=weight,
                        num_tries_per_arm: int=num_tries_per_arm) -> int:
    if any(num_draws < num_tries_per_arm):
        unpulled_arms = np.where(num_draws < num_tries_per_arm)[0]
        return np.random.choice(unpulled_arms)

    total_draws = sum(num_draws)
    num_arms = len(expected_rewards)
    uncertainty = np.sqrt(np.log(total_draws + num_arms) / (num_draws + num_arms))
    ucb = expected_rewards + weight * uncertainty

    max_reward_arms = np.where(ucb == max(ucb))[0]
    return np.random.choice(max_reward_arms)

# run experiment with policy
experiment = Experiment(testbed, upper_confidence_bound)
experiment.run_episode(num_steps=1000)
experiment.visualise()

md(f'''
Optimal arm found with **UCB** is **{experiment.expected_rewards.argmax()}**

Number of times each arm was **pulled**: {dict(zip(range(len(experiment.num_draws)), experiment.num_draws))}

**Average reward** collected is **{np.array(experiment.observed_rewards).mean()}**
''')

In [ ]:
average_experiments(upper_confidence_bound, weight=0.5, num_tries_per_arm=[1, 2, 10])

### Thompson Sampling
In this policy, we are going to put a prior on each arm and learn the distribution of the rewards for each pulled arm directly.

To keep things simple, we will look at a Beta/Bernoulli example. For that, we will make a few modifications to our setup.

We will start with redifining our mean sampling functions and using them to create a Bernoulli testbed.

In [ ]:
def arm_mean_sampler(size: int=1) -> float:
    return np.random.uniform(size=size)

def arm_reward_sampler(p: float, size: int=1) -> float:
    reward = np.random.binomial(n=1, p=p, size=size)
    if size == 1:
        return reward[0]
    return reward

testbed = TestBed(num_arms=num_arms, arm_mean_sampler=arm_mean_sampler, arm_reward_sampler=arm_reward_sampler)
sns.scatterplot(x=range(num_arms), y=testbed.mean_rewards, ax=plt.subplots(figsize=(15, 5))[1])
plt.ylim(0, 1)
plt.xlabel('Arm')
plt.ylabel('Mean reward')
plt.show()

md(f"Arm with **maximum mean reward** is arm **{testbed.mean_rewards.argmax()}**")


Now let's redefine how our experiments are conducted for the Beta/Bernoulli use case.

In [ ]:
# set initial Beta prior parameters
alpha = 0
beta = 0

class ThompsonExperiment(Experiment):
    
    def _init_fields(self) -> None:
        self.observed_rewards = []
        # alphas - first column, betas - second column
        self.beta_params = np.tile([alpha, beta], self.testbed.num_arms).reshape(self.testbed.num_arms, 2)
        self.observed_draws = []
        self.num_draws = np.zeros(self.testbed.num_arms, dtype=np.int32)
        self.total_draws = 0

    def step(self) -> Tuple[int, float]:
        chosen_arm = self.policy(self.beta_params)
        reward = self.testbed.draw_arm(chosen_arm)

        self.total_draws += 1
        self.observed_draws.append(chosen_arm)
        self.num_draws[chosen_arm] += 1
        self.observed_rewards.append(reward)
        if reward == 1:
            self.beta_params[chosen_arm, 0] += 1
        else:
            self.beta_params[chosen_arm, 1] += 1

        return chosen_arm, reward


Now let's implement the sampling policy. For Thompson sampling, we will use the prior parameters to sample expected rewards for each arm and select the arm with the highest sampled reward. Depending on the observed reward, we update the prior parameters for the respective arm.

In [ ]:
def beta_bernoulli_priors(beta_params: List[List[int]]) -> int:
    sampled_rewards = []
    for i in range(len(beta_params)):
        alpha, beta = beta_params[i]
        reward = scipy.stats.beta.rvs(alpha + 1, beta + 1)
        sampled_rewards.append(reward)

    max_reward_arms = np.where(np.array(sampled_rewards) == max(sampled_rewards))[0]
    return np.random.choice(max_reward_arms)

# run experiment with policy
experiment = ThompsonExperiment(testbed, beta_bernoulli_priors)
experiment.run_episode(num_steps=1000)
experiment.visualise(kind='scatter')

expected_means = np.array([scipy.stats.beta.mean(a + 1, b + 1) for a, b in experiment.beta_params])

md(f'''
Optimal arm found with **Thompson sampling** is **{expected_means.argmax()}**

Number of times each arm was **pulled**: {dict(zip(range(len(experiment.num_draws)), experiment.num_draws))}

**Average reward** collected is **{np.array(experiment.observed_rewards).mean()}**
''')